In [1]:
#load data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import sys
import h5py
import tensorflow.compat.v1 as tf
import tensorflow as tf2

check_data = h5py.File('indoor_flying_preprocessed.hdf5')

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

tf.config.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.set_floatx('float32') #for memory use

In [4]:
target_height = 260
target_width = 340

#shuffle data 
from random import shuffle
ind_list = [i for i in range(len(check_data['event_images']))]
shuffle(ind_list)
event_images = [check_data['event_images'][i] for i in ind_list]
depth_images = [check_data['depth_images_timed'][i] for i in ind_list]


def get_batches_fn(batch_size):
    # shuffle data
    ind_list = [i for i in range(len(check_data['event_images']))]
    shuffle(ind_list)
    event_images = [check_data['event_images'][i] for i in ind_list]
    depth_images = [check_data['depth_images_timed'][i] for i in ind_list]
    for batch_i in range(0, len(event_images), batch_size):
        inputs = []
        outputs = []
        for i in range(batch_i, batch_i+batch_size):
            inputs.append(event_images[i][0:target_height, 0:target_width, :])
            output_image = depth_images[i][0:target_height, 0:target_width].reshape((target_height, target_width, 1))
            output_image[np.isnan(output_image)] = 0.0
            outputs.append(output_image)

            cost_weights = np.copy(outputs)
            cost_weights[cost_weights>0] = 1.0

        yield np.array(inputs), np.array(outputs), np.array(cost_weights)


In [5]:
print(check_data['event_images'][10][0:target_height, 0:target_width, :].shape)

(260, 340, 3)


<KeysViewHDF5 []>

In [7]:
def NN_model(input_tensor):
    
    #Layer 1: Convolutional
    conv_1 = tf.layers.Conv2D(kernel_size=(10, 10), filters=64, strides=1, padding='same', 
                kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                kernel_regularizer=tf2.keras.regularizers.l2(), bias_regularizer=tf2.keras.regularizers.l2(),
                name='conv_1')(input_tensor)

    #Layer 2: Convolutional
    conv_2 = tf.layers.Conv2D(kernel_size=10, filters=64, strides=1, padding='same', 
                kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                kernel_regularizer=tf2.keras.regularizers.l2(), bias_regularizer=tf2.keras.regularizers.l2(), activation='relu',
                name='conv_2')(conv_1)
    

    #pooling function
    pool_1 = tf.layers.MaxPooling2D(pool_size=2, strides=2, padding='same', name='pool_1')(conv_2)

    #Layer 3: Convolutional
    conv_3 = tf.layers.Conv2D(kernel_size=10, filters=128, strides=1, padding='same', 
                kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                kernel_regularizer=tf2.keras.regularizers.l2(), bias_regularizer=tf2.keras.regularizers.l2(),
                name='conv_3')(pool_1)

    #Layer 4: Convolutional
    conv_4 = tf.layers.Conv2D(kernel_size=10, filters=128, strides=1, padding='same', 
                kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                kernel_regularizer=tf2.keras.regularizers.l2(), bias_regularizer=tf2.keras.regularizers.l2(), activation='relu',
                name='conv_4')(conv_3)

    #pooling function
    pool_2 = tf.layers.MaxPooling2D(pool_size=2, strides=2, padding='same', name='pool_2')(conv_4)


    #1x1 convolutions 1
    conv1x1_1 = tf.layers.Conv2D(kernel_size=1, filters=256, strides=1, padding='same',
                    kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                    kernel_regularizer=tf2.keras.regularizers.l2(), 
                    name='conv1x1_1')(pool_2)
    
    tanh_1 = tf2.keras.activations.tanh(conv1x1_1)

    #1x1 convolutions 2
    conv1x1_2 = tf.layers.Conv2D(kernel_size=1, filters=512, strides=1, padding='same',
                    kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                    kernel_regularizer=tf2.keras.regularizers.l2(),
                    name='conv1x1_2')(tanh_1)

    
    tanh_2 = tf2.keras.activations.tanh(conv1x1_2)


    #Deconvolution 1
    deconv_1 = tf2.keras.layers.Conv2DTranspose(kernel_size=10, filters=128, strides=2, padding='same',
                    kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                    kernel_regularizer=tf2.keras.regularizers.l2(),  activation='relu',
                    name='deconv_1')(tanh_2)
                    
    #Skipping layer 1
    skip_1 = tf2.keras.layers.Add(name='skip_1')([deconv_1, conv_4])

    #Deconvolution 1
    deconv_2 = tf2.keras.layers.Conv2DTranspose(kernel_size=10, filters=64, strides=2, padding='same',
                    kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                    kernel_regularizer=tf2.keras.regularizers.l2(), activation='relu',
                    name='deconv_2')(skip_1)
    
    #Skipping layer 2
    skip_2 = tf2.keras.layers.Add(name='skip_2')([deconv_2, conv_2])

    #Final layer
    output_layer = tf.layers.Conv2D(kernel_size=10, filters=1, strides=1, padding='same',
                    kernel_initializer=tf2.keras.initializers.GlorotNormal(), bias_initializer=tf2.keras.initializers.GlorotNormal(),
                    kernel_regularizer=tf2.keras.regularizers.l2(), 
                    name='output_layer')(skip_2)

    return output_layer






In [8]:
def optimize(nn_last_layer, correct_value, cost_weights, learning_rate):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label 
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """    
    #reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) #regularization loss
    #mse_loss = tf.keras.losses.MeanSquaredError()(tf2.math.multiply(correct_value, correct_value), tf2.math.multiply(nn_last_layer, correct_value)) #cross entropy
    #overall loss combining cross entropy and regularization
    #overall_loss = tf2.math.add(1.0*tf2.reduce_sum(reg_loss), mse_loss, name='total_loss')

    mse_loss = tf.keras.losses.MeanSquaredError()(correct_value, tf2.math.multiply(nn_last_layer, cost_weights)) #cross entropy
  
    #obtain training operation
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, epsilon = 0.000001) #Note default value of epsilon 1e-8 results in instability after few epochs
   
    #clip the gradients
    gvs = optimizer.compute_gradients(mse_loss)
    #capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
    training_operation = optimizer.apply_gradients(gvs)

    return nn_last_layer, training_operation, mse_loss

In [9]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, loss_function, input_image,
             depth_image, costs_weights, learning_rate, base_learning_rate):
    """
    Train neural network and print    out the loss during training.
  param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param output_image: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    #initialize variables
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    print()
    scaling_rate = 1

    for i in range(epochs):
        print("EPOCH {} ...".format(i+1))
        if i > 1:
          scaling_rate = 0.5 * scaling_rate
        for image, output_image, costs in get_batches_fn(batch_size):

            optimizer, loss = sess.run([train_op, loss_function], 
                               feed_dict={input_image: image, depth_image: output_image, costs_weights: costs, learning_rate: scaling_rate*base_learning_rate})
            print("Loss: =")
            print(loss)
            print()

In [10]:
epochs = 7
batch_size = 5
base_learning_rate = 0.00001


tf.compat.v1.disable_eager_execution()
input_image = tf.placeholder(tf.float32, [None, None, None, 3], name='input_image')
depth_image = tf.placeholder(tf.float32, [None, None, None, 1], name='depth_image')
costs_weights = tf.placeholder(tf.float32, [None, None, None, 1], name='cost_weights')
learning_rate = tf.placeholder(tf.float32, shape=(), name='learning_rate')

sess = tf.Session()

# Build NN using load_vgg, layers, and optimize function
nn_last_layer = NN_model(input_image)

logits, train_op, mse_loss = optimize(nn_last_layer, depth_image, costs_weights, learning_rate)

# Train NN using the train_nn function
train_nn(sess, epochs, batch_size, get_batches_fn, train_op, mse_loss, input_image,
        depth_image, costs_weights, learning_rate, base_learning_rate)
        
# save trained model
saver = tf.train.Saver()
saver.save(sess, './model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Training...

EPOCH 1 ...
Loss: =
3.8189669

Loss: =
5.6658134

Loss: =
4.9568863

Loss: =
3.181001

Loss: =
3.6343286

Loss: =
2.8817313

Loss: =
2.4874167

Loss: =
1.9631627

Loss: =
2.598578

Loss: =
2.8947523

Loss: =
2.286173

Loss: =
2.3715034

Loss: =
1.8515717

Loss: =
2.384304

Loss: =
2.1634274

Loss: =
1.9007412

Loss: =
1.5976202

Loss: =
1.641837

Loss: =
1.7707744

Loss: =
1.168307

EPOCH 2 ...
Loss: =
1.527378

Loss: =
1.5142325

Loss: =
1.558737

Loss: =
1.1203138

Loss: =
1.1809367

Loss: =
1.1682237

Loss: =
1.2975161

Loss: =
1.4945607

Loss: =
1.1636338

Loss: =
1.9034463

Loss: =
1.2491916

Loss: =
1.3484462

Loss: =
1.1430475

Loss: =
1.3737319

Loss: =
1.398198

Loss: =
1.6138952

Loss: =
1.2676265

Loss: =
1.2895448

Loss: =
1.394231

Loss: =
1.4365782

EPOCH 3 ...
Loss: =
1.3877321

Loss: =
1.391662

Loss: =
1.2734365

Loss: =
1.2197832

Loss: =
1.4701116

Loss: =
1.2070272

Loss: =

KeyboardInterrupt: 

In [25]:
saver = tf.train.Saver()
saver.save(sess, './model')

FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Error while reading resource variable output_layer/kernel/Adam_1 from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/output_layer/kernel/Adam_1)
	 [[node output_layer/kernel/Adam_1/Read/ReadVariableOp (defined at <ipython-input-8-37efa903502c>:21) ]]
	 [[conv_2_1/bias/Adam_1/Read/ReadVariableOp/_97]]
  (1) Failed precondition: Error while reading resource variable output_layer/kernel/Adam_1 from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/output_layer/kernel/Adam_1)
	 [[node output_layer/kernel/Adam_1/Read/ReadVariableOp (defined at <ipython-input-8-37efa903502c>:21) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'output_layer/kernel/Adam_1/Read/ReadVariableO

In [26]:
plt.imshow(check_data['depth_images_timed'][65][0:target_height, 0:target_width], cmap='gray')

NameError: name 'check_data' is not defined

In [27]:
sample_image = check_data['event_images'][65][0:target_height, 0:target_width, :]
est_depth = sess.run([nn_last_layer], feed_dict={input_image: sample_image.reshape(1, 260, 340, 3)})

plt.imshow(est_depth[0].reshape(260, 340), cmap='gray')

NameError: name 'check_data' is not defined

In [28]:
plt.imshow(sample_image)

NameError: name 'sample_image' is not defined

In [29]:
print(depth_images[49][0:target_height, 0:target_width])

NameError: name 'depth_images' is not defined